<a href="https://colab.research.google.com/github/ShiranGiladi/Cloud-Computing/blob/tirgul6-7/pythonGameTirgul6_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import IPython
from google.colab import output
import json
from IPython.display import HTML, clear_output
from firebase import firebase
import random
# import users data from the DB
FBconn = firebase.FirebaseApplication('https://tirgul6-41853-default-rtdb.firebaseio.com/',None)
res=FBconn.get('/users/',None)
resQuestion=FBconn.get('/question/',None)

global userInPlayFirstName, userInPlayLastName, userInPlayPassword,questionNum, userInPlayuserName
questionNum=0

# login page
loginHTML = \
'''
<div id="screen">
  <h2 id="title">Log In</h2>
  <br><input type="text" id="userName" name="userName" placeholder="userName" required></input>
  <br><br><input type="password" id="password" name="password" placeholder="password" required></input>
  <div id="msg"></div>
  <br><br><button id='button'>Connect</button>
</div>
<script>
  document.querySelector('#button').onclick = () => {
    google.colab.kernel.invokeFunction('connect', [userName.value, password.value], {kwarg1: true});
  };
</script>
'''

# admin page
adminHTML = \
'''
<form id="screen">
  <h2 id="title">Enter New Question</h2>
  <br><input type="text" id="question" name="question" placeholder="question" required></input>
  <br><br><input type="text" id="correctAnswer" name="correctAnswer" placeholder="correctAnswer" required></input>
  <br><br><input type="text" id="worngAnswer1" name="worngAnswer1" placeholder="worngAnswer1" required></input>
  <br><br><input type="text" id="worngAnswer2" name="worngAnswer2" placeholder="worngAnswer2" required></input>
  <br><br><input type="text" id="worngAnswer3" name="worngAnswer3" placeholder="worngAnswer3" required></input>
  <div id="msg"></div>
  <br><br><button id='addButton'>Add question</button>
  <br><br><button id='logoutButton'>Logout</button>
</form>
<script>
  document.querySelector('#addButton').onclick = () => {
    google.colab.kernel.invokeFunction('addQuestion', [question.value, correctAnswer.value, worngAnswer1.value, worngAnswer2.value, worngAnswer3.value], {kwarg1: true});
  };
  document.querySelector('#logoutButton').onclick = () => {
    google.colab.kernel.invokeFunction('logout', [], {});
  };
</script>
'''

# Daniel
# registration page
registerHTML = \
'''
<div id="registrationScreen">
  <h2 id="title">Sign up</h2>
  <br><input type="text" id="userName" name="userName" placeholder="userName" required></input>
  <br><br><input type="text" id="firstName" name="firstName" placeholder="firstName" required></input>
  <br><br><input type="text" id="lastName" name="lastName" placeholder="lastName" required></input>
  <br><br><input type="password" id="password" name="password" placeholder="password" required></input>
  <div id="msg"></div>
  <br><br><button id='button'>Register</button>
</div>
<script>
  document.querySelector('#button').onclick = () => {
    google.colab.kernel.invokeFunction('register', [userName.value, firstName.value, lastName.value, password.value], {kwarg1: true});
  };
</script>
'''

# game page
gameHTML = \
'''
<div id='question'></div>
<div id='answers'></div>
'''

# Daniel
def register(userName, firstName, lastName, password, kwarg1=None):
    res = FBconn.get('/users/', None)
    new_key = str(len(res))
    if newUserExist(res, userName):
        with output.redirect_to_element('#msg'):
            display(HTML('<h4>User Already Registered!</h4>'))
    else:
        data_to_upload = {
            'firstName': firstName,
            'userName': userName,
            'lastName': lastName,
            'password': password,
            'admin': "no",
            'score': "0",
            'countCorrectAnswer': "0",
            'currentQuestionInARow': "0",
            'questionsThatBeenAnswerd': ["0"]
        }
        FBconn.put('/users/', name=new_key, data=data_to_upload)

        print(f"New user {userName} registered with key {new_key}")

def newUserExist(res, userName, kwarg1=None):
    for key in res:
        if(key['userName'] == userName ):
            return True
    return False

def connectCheck(userName, password, kwarg1=None):
  if(user_exist_check(userName, password)):
    clear_output()
    if(admin_check(userName)):
      display(HTML(adminHTML))
    else:
      display(HTML(gameHTML))
      load_question();
  else:
    with output.redirect_to_element('#msg'):
      display(HTML('<h4>Try Again!</h4>'))

#raeed change
def user_exist_check(userName, password):
  for key in res:
    if(key['userName'] == userName  and key['password'] == password):
      global userInPlayuserName,userInPlayPassword
      userInPlayuserName = userName
      userInPlayPassword = password
      return True
  return False

def admin_check(userName):
  for key in res:
    if(key['userName'] == userName and key['admin'] == "yes"):
      return True;
  return False;

def logout():
  clear_output()
  display(HTML(loginHTML))

def addQuestion(question, correctAnswer, worngAnswer1, worngAnswer2, worngAnswer3, kwarg1=None):
  while(True):
    data_to_upload = {
      'q' : question,
      'correctAnswer' : correctAnswer,
      'worngAnswer1' : worngAnswer1,
      'worngAnswer2' : worngAnswer2,
      'worngAnswer3' : worngAnswer3,
      'id': len(resQuestion)
    }
    FBconn.post('/question/',data_to_upload)

def getQuestionsFromDB(questionNum):
  j=0
  questions=FBconn.get('/question/',None)
  for question in questions:
    if(j==questionNum):
      return question
    j+=1
  return question

#raeed
def load_question():
    clear_output()
    global questionNum
    question = getQuestionsFromDB( questionNum)
    questionNum += 1
    if not question:
        display(HTML('<h2>Finished</h2><br><br>'))
        return

    display(HTML(f"<h2>{question['q']}</h2><br><br>"))
    # Create a list of answer options
    answer_options = [question['correctAnswer'], question['worngAnswer1'], question['worngAnswer2'], question['worngAnswer3']]
    # Shuffle the order of the answer options
    random.shuffle(answer_options)
    # Display the shuffled answer options
    for answer in answer_options:
        display(HTML(f'<input type="radio" name="answer" value="{answer}"> {answer}</input><br><br>'))
    # Add a submit button
    display(HTML('<button id="check-btn" onclick="submitAnswer()">Check</button>'))
    #display(HTML(f'<button id="nextQuestion" onclick="nextQuestionF({questionNum})">Next</button>'))
    display(HTML(f'<button id="nextQuestion">Next</button>'))
    # Add a JavaScript function to handle the submit button click
    display(HTML(f'''
        <script>
           document.getElementById('nextQuestion').style.display = 'none'; 
        
        nextQuestion.addEventListener('click', function() {{
           document.getElementById('answer-result').innerHTML = '<h3>hi12.</h3>';
           google.colab.kernel.invokeFunction('load_question', []);
 
           document.getElementById('answer-result').innerHTML = '<h3>hi12.</h3>';

        }});           
            function submitAnswer() {{
                const radios = document.getElementsByName('answer');
                let selectedAnswer = '';

                for (const radio of radios) {{
                    if (radio.checked) {{
                        selectedAnswer = radio.value;
                        break;
                    }}
                }}
                if (selectedAnswer==='') {{
                   document.getElementById('answer-result').innerHTML = '<h3>select an answer</h3>';
                   return;

  
                }}

                if (selectedAnswer === '{question['correctAnswer']}') {{
                    console.log('Correct!');
                    document.getElementById('answer-result').innerHTML = '<h3>Correct!</h3>';
                    document.getElementById('check-btn').style.display = 'none';
                    document.getElementById('nextQuestion').style.display = 'block';


                }} else {{
                    console.log('Incorrect.');
                    document.getElementById('answer-result').innerHTML = '<h3>Incorrect.</h3>';
                    document.getElementById('check-btn').style.display = 'none';
                    document.getElementById('nextQuestion').style.display = 'block';


                }}

                // Hide the check button

            }}
        </script>
        <div id="answer-result"></div>
    '''))


#display(HTML(gameHTML))
#def startPlaying():
#    load_question()
#startPlaying()

display(HTML(loginHTML))
output.register_callback('load_question', load_question)
output.register_callback('connect', connectCheck)
output.register_callback('addQuestion', addQuestion)
output.register_callback('logout', logout)